# Análise e Transformação de Dados
#### Licenciatura em Engenharia Informática - Universidade de Coimbra
#### 2020/2021

## Projeto final

### Autores:
- João Filipe Guiomar Artur, 2019217853
- Samuel Santos Carinhas,
- Sancho Amaral Simões, 2019217590

## 1. Introdução

<div style="text-align: justify">&nbsp;&nbsp;&nbsp;&nbsp;O presente projeto apresenta como finalidade a aplicação de alguns dos conhecimentos adquiridos pelos autores no âmbito da cadeira de <strong>Análise e Transformação de Dados</strong>, nomeadamente a análise de dados originalmente facultados no domínio temporal e a sua posterior transposição para o domínio da frequência / domínio misto (frequência/tempo), com recurso a ferramentas matemáticas como a transformada de Fourier e as suas variantes (DFT, STFT, ...). 
<br>&nbsp;&nbsp;&nbsp;&nbsp;Para o desenvolvimento deste projeto, foi fornecido um dataset no qual constam valores recolhidos por acelerómetros de smartphones nas três dimensões (x, y e z). Estes valores são provenientes de várias sequências de movimentos executados por um grupo de 30 voluntários. Nas referidas sequências encontram-se movimentos de tipo estático, dinâmico e de transição. A terminologia utilizada para caracterizar os grupos de movimentos é bastante autoxplicativa, como se vê em baixo. 
    
<ul>
     <li> 
         <h5> Dinâmicos </h5>
         <ol>
             <li> Andando (WALKING) </li>
             <li> Subindo Escadas (WALKING_UPSTAIRS) </li>
             <li> Descendo Escadas (WALKING_DOWNSTAIRS) </li>
         </ol>
     </li>
     <li> 
         <h5> Estáticos </h5>
         <ol>
             <li> Sentado (SITTING) </li>
             <li> De pé (STANDING) </li>
             <li> Deitado (LAYING) </li>
         </ol>
     </li>
    <li> 
         <h5> Transição </h5>
         <ol>
             <li> De pé -> Sentado (STAND_TO_SIT) </li>
             <li> Sentado -> De pé (SIT_TO_STAND) </li>
             <li> Sentado -> Deitado (SIT_TO_LIE) </li>
             <li> Deitado -> Sentado (LIE_TO_SIT) </li>
             <li> De pé -> Deitado (STAND_TO_LIE) </li>
             <li> Deitado -> De pé (LIE_TO_STAND) </li>
         </ol>
     </li>  
</ul>
    
&nbsp;&nbsp;&nbsp;&nbsp;
</div>

## 2. Workspace

&nbsp;&nbsp;&nbsp;&nbsp;Para maior legibilidade / facilidade de manutenção de código, foi criado um workspace inicial que contém algumas constantes utilizadas ao longo do desenvolvimento do projeto.

In [3]:
% region constants
X_AXIS = 1;
Y_AXIS = 2;
Z_AXIS = 3;
DATASETS_DIR = 'datasets';
LABELS_PATH = 'labels.txt';
COLORS = {'2f4f4f', '7f0000', '008000', '00008b', 'ff8c00', 'ffff00', '00ff00', '00ffff', 'ff00ff', '1e90ff', 'ffe4b5', 'fd4002'};
ACTIVITIES = ["WALK", "WALK\_UP", "WALK\_DOWN", "SIT", "STAND", "LAY", "STAND\_SIT", "SIT\_STAND", "SIT\_LIE", "LIE\_SIT", "STAND\_LIE", "LIE\_STAND"];
FS = 50;
THRESHOLD = 0.25;
% end region constants

## 3. Importação de dados

&nbsp;&nbsp;&nbsp;&nbsp; Os datasets fornecidos foram importados através da utilização da função load_datasets. Esta percorre todos os ficheiros .txt no diretório especificado e lê, através da função dlmread, os valores lidos pelos acelerómetros, utilizando como delimitador o espaço.

In [9]:
%%file load_datasets.m

function datasets = load_datasets(directory)
    dataset_files = dir([directory '/*.txt']);
    len = length(dataset_files);
    datasets = cell(1, len);
    
    for i = 1:len
        datasets(i) = {dlmread([directory '/' dataset_files(i).name], ' ')};
    end
end

Created file 'C:\Users\smart\Documents\GitRepos\Actionalizer\load_datasets.m'.


In [10]:
labels = dlmread(LABELS_PATH, ' ');
datasets = load_datasets(DATASETS_DIR);

## 4. Representação dos sinais

In [13]:
%%file plot_datasets.m


function plot_datasets(datasets, fs, labels, activities)
   len = length(datasets);
   label_i = 1;
   ts = 1/fs;
   
   figure;
   
   for i = 1:len
       dataset = cell2mat(datasets(i));
       init_label = label_i;
       
       for k = 1:3
           label_i = init_label;
           subplot(3, 1, k);
           plot(ts/60:ts/60:(length(dataset) * ts)/60, dataset(:, k), 'k');
           y_lbl = sprintf('ACC angle - %s (m/s^2)', get_axis_name(k));
           ylabel(y_lbl);
           
           xlabel('Time (min)')
           
           hold on
           
           while 1
               exp = labels(label_i, 1);

               if exp ~= i
                    break;
               end

               user = labels(label_i, 2);
               act = labels(label_i, 3);
               start = labels(label_i, 4);
               finish = labels(label_i, 5);
               x = start * ts/60:ts/60:finish * ts/60;
               y_point = mean(dataset(start:finish, k)) + 0.7 * (-1)^label_i;
               plot(x, dataset(start:finish, k));
               text(start*ts/60, y_point, activities(act),'Fontsize', 7);
               label_i = label_i + 1;
           end
           
           plt_title = sprintf('Experience %d, User %d', exp, user);
           sgtitle(plt_title);
           
           hold off;
       end
       
       pause(2);
   end
end

Created file 'C:\Users\smart\Documents\GitRepos\Actionalizer\plot_datasets.m'.


In [ ]:
plot_datasets(datasets, FS, labels, ACTIVITIES);

![Title](dataset1.fig)